In [5]:
from PIL import Image
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [6]:
output_path = 'new/mixup/'
penguin_path = 'new/train/penguin/'
turtle_path = 'new/train/turtle/'
train_annotations = pd.read_json('new/train_annotations')
#train_annotations = train_annotations[train_annotations['image_id'] < 500]
penguin_order = train_annotations[train_annotations['category_id'] == 0]['image_id'].to_numpy()
penguin_order = np.random.permutation(penguin_order)
turtle_order = train_annotations[train_annotations['category_id'] == 1]['image_id'].to_numpy()
turtle_order = np.random.permutation(turtle_order)

In [11]:
# Generate mixup between two images
def mixup(image1, image2):
    # Generate image weight (minimum 0.4 and maximum 0.6)
    lam = np.clip(np.random.beta(0.2, 0.2), 0.4, 0.6)
    lam=round(lam,2)
    #print(f'lambda: {lam}')

    # Weighted Mixup
    mixedup_image = lam * image1 + (1 - lam) * image2
    return mixedup_image, lam

df = []
image_id = 5000
for i, v in enumerate(tqdm(penguin_order)):
    p = Image.open(penguin_path + 'image_id_' + str(v) + '.jpg')
    v2 = turtle_order[i]
    t = Image.open(turtle_path + 'image_id_' + str(v2) + '.jpg')
    m, prob = mixup(p, t)
    m = np.asarray(m, dtype=np.uint8)
    im = Image.fromarray(m)
    im.save(output_path + "mixup_image_id_" + str(image_id) + ".jpg")
    p_bbox = train_annotations[train_annotations['image_id'] == int(v)].values[0][2]
    p_area = train_annotations[train_annotations['image_id'] == int(v)].values[0][3]
    t_bbox = train_annotations[train_annotations['image_id'] == int(v2)].values[0][2]
    t_area = train_annotations[train_annotations['image_id'] == int(v2)].values[0][3]
    d = {"image_id": image_id, "category_id": [prob, 1 - prob], "bbox": [p_bbox, t_bbox],
         "area": [p_area, t_area]}
    image_id += 1
    df.append(d)

file_path = 'new/mixup_annotation.json'
with open(file_path, 'w') as file:
    json.dump(df, file)

100%|██████████| 2250/2250 [00:24<00:00, 93.10it/s]
